In [3]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from keras import layers
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_curve, auc

2024-04-15 14:53:11.182391: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-15 14:53:11.263068: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-15 14:53:11.264158: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-15 14:53:12.201397: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

In [5]:
train_ds = train.flow_from_directory('/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/train',
                                     target_size = (224,224), batch_size = 10)

test_ds = test.flow_from_directory('/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/test',
                                  target_size = (224,224), batch_size = 10)

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [6]:
# Set up your data generator using flow_from_directory
train_generator = train.flow_from_directory(
    '/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/train',
    target_size=(224, 224),  # Adjust the target size according to EfficientNet-B0 input shape
    batch_size=10,
    class_mode='binary')  # Assuming you're doing multi-class classification

# Grab the class labels
train_label = list(train_generator.class_indices.keys())
print("Class Labels:", train_label)

Found 251 images belonging to 3 classes.
Class Labels: ['Covid', 'Normal', 'Viral Pneumonia']


In [7]:
# Set up your data generator using flow_from_directory
test_generator = test.flow_from_directory(
    '/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/test',
    target_size=(224, 224),  # Adjust the target size according to EfficientNet-B0 input shape
    batch_size=10,
    class_mode='binary')  # Assuming you're doing multi-class classification

# Grab the class labels
test_label = list(test_generator.class_indices.keys())
print("Class Labels:", test_label)

Found 66 images belonging to 3 classes.
Class Labels: ['Covid', 'Normal', 'Viral Pneumonia']


# MobileNet

In [10]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import matplotlib.pyplot as plt

# Set up your ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)  # You can add other preprocessing steps here

# Set up your data generator using flow_from_directory for validation data
validation_generator = train.flow_from_directory(
    '/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/train',
    target_size=(224, 224),  # Adjust the target size according to EfficientNet-B0 input shape
    batch_size=10,
    class_mode='binary',
    shuffle=False) 

Found 251 images belonging to 3 classes.


In [11]:
# Load MobileNet without the top layer
base_model = MobileNet(weights='imagenet', include_top=False)

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
history = model.fit(train_generator, 
                    epochs=10, 
                    batch_size=10, 
                    validation_data=validation_generator)

Epoch 1/10


2024-04-15 14:53:39.064024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


ValueError: in user code:

    File "/home/efazh/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/efazh/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/efazh/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/efazh/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/efazh/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/home/efazh/.local/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p

In [ ]:
# Plot loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Over Time')
plt.legend()
plt.show()

In [ ]:
# Plot validation accuracy
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy Over Time')
plt.legend()
plt.show()

# VGG19

In [ ]:
from tensorflow.keras.applications import VGG19

# Set up your ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)  # You can add other preprocessing steps here

# Set up your data generator using flow_from_directory
train_generator = train.flow_from_directory(
    '/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/train',
    target_size=(224, 224),  # Adjust the target size according to input shape
    batch_size=10,
    class_mode='binary')  # Assuming you're doing multi-class classification

# Load VGG19 without the top layer
base_model = VGG19(weights='imagenet', include_top=False)

# Grab the class labels
class_labels = list(train_generator.class_indices.keys())
print("Class Labels:", class_labels)

In [ ]:
from sklearn.metrics import roc_curve, auc

# Get the predicted probabilities for the validation set
validation_generator = train.flow_from_directory(
    '/mnt/c/users/efazh/DataProblems/Datasets/ComputerVision/Covid19-dataset/train',
    target_size=(224, 224),  # Adjust the target size according to EfficientNet-B0 input shape
    batch_size=10,
    class_mode='binary',
    shuffle=False) 

# Predict probabilities using the model
probabilities = model.predict(validation_generator)

# Get true labels
true_labels = validation_generator.classes

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(true_labels, probabilities[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve
plt.figure(figsize=(8, 6))
for i in range(3):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve (class {i}) (AUC = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()
